In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data=pd.read_csv("tmdb_5000_movies.csv")
data.head()
data.shape
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [3]:
########데이터 전처리##########
#추천시스템 동작 : 기존에 이미 시청한 영화의 제목을 입력 -> 영화와 비슷한 영화를 추천
#                                                    컨텐츠기반추천
data=data[['id','genres', 'vote_average','vote_count', 'popularity', 
      'title', 'keywords', 'overview']]



"""
필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?
문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)
feature engineering(특성공학, )

xdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)
국어, 영어, 수학, 언어능력(50점만점)   수능점수
80     90   100 ,  40   400점
         ...
100명 학생 점수 data를 수집

파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수

x변수들 -> 모델 -> y예측값(y hat)
90 100 95 -> ???예측

타이타닉 data 
승객명단          성별(파생)
Mrs. 이름...        0
Mr. 이름...           1
Dr. ...

"""

'\n필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?\n문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)\nfeature engineering(특성공학, )\n\nxdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)\n국어, 영어, 수학, 언어능력(50점만점)   수능점수\n80     90   100 ,  40   400점\n         ...\n100명 학생 점수 data를 수집\n\n파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수\n\nx변수들 -> 모델 -> y예측값(y hat)\n90 100 95 -> ???예측\n\n타이타닉 data \n승객명단          성별(파생)\nMrs. 이름...        0\nMr. 이름...           1\nDr. ...\n\n'

In [4]:
#data['vote_count'].quantile(0.9)#1838
#상위 10% 지점에 해당되는 vote_count 컬럼 값이 1838 이다.
tmp_m=data['vote_count'].quantile(0.89)#1683
data['vote_count']>=tmp_m
tmp_data=data.copy()[data['vote_count']>=tmp_m]
tmp_data.shape

m=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=m]
#vote_count값에 대해 대략 500등 안에 들어가는 데이터를 추출

#data.info()
#수치데이터(연속형(점수,온도), 범주형(성별,혈액형,학점))
#연속형 수치데이터 -> 구간 -> 범주화

In [5]:
data.shape
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [6]:
c=data['vote_average'].mean()
c #6.96
m #1838

1838.4000000000015

### Bayesian Estimator

In [7]:
#(WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*c)
    
data['score']=data.apply(weighted_rating, axis=1)    

## 데이터 전처리
### 장르, 키워드

In [8]:
data.head()
data[['genres', 'keywords']].head()
#리스트 내부에 딕셔너리 구조로 저장

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [9]:
#eval()
listStr='[1,2,3,4,5]'
listStr

'[1,2,3,4,5]'

In [10]:
import ast
listData=ast.literal_eval(listStr)
listData

[1, 2, 3, 4, 5]

In [11]:
data['genres']=data['genres'].apply(ast.literal_eval)

In [12]:
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [13]:
data[['genres', 'keywords']]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
...,...,...
4291,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...","[{'id': 1366, 'name': 'shotgun'}, {'id': 13000..."
4300,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...","[{'id': 2052, 'name': 'traitor'}, {'id': 6099,..."
4302,"[{'id': 37, 'name': 'Western'}]","[{'id': 801, 'name': 'bounty hunter'}, {'id': ..."
4337,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'..."


In [14]:
data.iloc[0]['genres']
# 모든 데이터의 장르 이름을 출력
#     genres
# 0  Action
#    Adventure
#      ...
# 1  Adventure
#     ...

data.iloc[1]['keywords']
#apply함수 활용

[{'id': 270, 'name': 'ocean'},
 {'id': 726, 'name': 'drug abuse'},
 {'id': 911, 'name': 'exotic island'},
 {'id': 1319, 'name': 'east india trading company'},
 {'id': 2038, 'name': "love of one's life"},
 {'id': 2052, 'name': 'traitor'},
 {'id': 2580, 'name': 'shipwreck'},
 {'id': 2660, 'name': 'strong woman'},
 {'id': 3799, 'name': 'ship'},
 {'id': 5740, 'name': 'alliance'},
 {'id': 5941, 'name': 'calypso'},
 {'id': 6155, 'name': 'afterlife'},
 {'id': 6211, 'name': 'fighter'},
 {'id': 12988, 'name': 'pirate'},
 {'id': 157186, 'name': 'swashbuckler'},
 {'id': 179430, 'name': 'aftercreditsstinger'}]

In [15]:
def name_to_string(columns):
    for col in columns:
        data[col] = (
            data[col].apply(lambda x: [d['name'] for d in x])
                     .apply(lambda x: " ".join(x))
            )

name_to_string(['genres', 'keywords'])

In [16]:
data[['genres', 'keywords']].head()

,genres,keywords
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...
2,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...


### 벡터화: 장르(문자열) 숫자로 변환 -> 벡터화 작업  
* **BoW**(Bag of Words, 단어가방): 문서를 구성하는 모든 단어들에 대해 표시하는 방법
* **Corpus**(코퍼스): 말뭉치, 단어(도메인) 사전

문서1: 나는 한국 사람  
문서2: 나는 미국 사람  
문서3: 나는 중국 사람  

위 문서의 corpus = {나는, 한국, 미국, 중국, 사람}  

토지소설 -> 원고 -> 토지 코퍼스: {토지 소설에 등장하는 단어들}
태백산맥 소설 -> 원고 -> 태백산맥 단어들 중에는 토지 코퍼스에 있는 단어도 있지만,  
없는 단어도 있을 것이다(예, '지리산' 단어는 토지에 등장하지 않음 -> OOV라고 함)  

#### BoW
전체 문서{d1, ..., dn}을 구성하는 단어장(vocabulary, {t1, t2, ..., tm})을 만든 후     
각각의 문서(n개)에 대해서 단어장에 있는 각각의 단어가 문서 내에 포함되어 있는 지, 아닌 지를 표시하는 방법.

In [ ]:
from sklearn.feature_extraction import DictVectorizer

#DictVectorizer: 단어의 빈도 정보가 담긴 딕셔너리로부터 BOW 벡터를 생성
v = DictVectorizer()
D = [{'A': 1, 'B': 2}, {'B':3, 'C': 1}] # 단어의 빈도 정보가 담긴 딕셔너리, 문서1, 2
x = v.fit_transform(D)
x

### Dense vs. Sparse Matrix
* **Sparse(희소행렬)**: 값이 대부분 0으로 채워진 행렬
* **Dense(밀집행렬)**: 대부분의 값이 0이 아닌 값으로 채워진 행렬

### DictVectorizer()
각 단어의 수를 세어놓은 사전에서 BOW 인코딩 벡터를 만든다.

### CountVectorizer()
문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만든다.

In [ ]:
v = DictVectorizer(sparse=False)
x = v.fit_transform(D)
x

In [ ]:
v.feature_names_

In [ ]:
# {'C': 4, 'D': 2} 딕셔너리를 v변수에 저장된 벡터화 방식으로 변환(bow)해라
v.transform({'C': 4, 'D': 2})

In [ ]:
# CountVectorizer(): 문서 -> 단어 토큰 리스트 -> 단어 출현 빈도 -> bow 벡터 변환
corpus = [ # 해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]

cv = CountVectorizer()
cv.fit(corpus)
cv.vocabulary_ # 총 10개의 단어가 corpus(도메인)에 존재

In [ ]:
cv.transform(corpus).toarray() # 딕셔너리에 있는 VALUE가 인덱스로

In [ ]:
# 새로운 문서를 입력받아 BOW생성
print(cv.transform(['Something completely new', 'This is the third doc']).toarray())

### stop_words: 불용어
자주 등장하지만 실제 의미 분석을 하는 데 기여가 없는 단어들 토큰을 거하는 작업  
* I, my, me, over, 조사, 접미사, 등

In [ ]:
cv = CountVectorizer(stop_words=['is', 'the'])
cv.fit(corpus)
print(cv.vocabulary_)
print(cv.transform(corpus).toarray())

In [ ]:
cv = CountVectorizer(stop_words='english')
cv.fit(corpus)
cv.vocabulary_

#### 미리 만들어진 stop_words 들 확인

In [ ]:
cv.get_stop_words()

In [ ]:
cv = CountVectorizer(stop_words='english', analyzer='char') # analyzer와 stop_words의 순서가 상관이 있음
cv.fit(corpus)
cv.vocabulary_

In [ ]:
cv = CountVectorizer(token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

## n-gram (Word2vec)
* 모노그램: 토큰 하나만 단어, 바이그램: 두개 토큰을 하나의 단어로 사용
* bigram: This is the first document
```python
['This is', 'is the', 'the first', 'first document']
```

In [ ]:
cv = CountVectorizer(ngram_range=(2, 2)) # [start, end] 
cv.fit(corpus)
cv.vocabulary_

In [ ]:
cv = CountVectorizer(ngram_range=(1, 2), token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

### Frequency(빈도수) 기반
`[min_df, max_df)`


In [ ]:
cv = CountVectorizer(min_df=1, max_df=3)
cv.fit(corpus)
cv.vocabulary_

## 장르 벡터화

In [ ]:
count_vector = CountVectorizer(ngram_range=(1, 3))
c_vector_genres = count_vector.fit_transform(data['genres']) #fit() transform() 한번에

In [ ]:
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

In [ ]:
count_vector = CountVectorizer(ngram_range=(1, 3))
count_vector.fit(data['genres'])
len(count_vector.get_feature_names())

In [ ]:
count_vector.get_feature_names()